### Setup

In [ ]:
%pip install transformers
%pip3 install torch --index-url https://download.pytorch.org/whl/cu126

In [ ]:
%pip install --upgrade ipywidgets

### Run

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from transformers import pipeline

In [ ]:
chatbot = pipeline(task="text-generation", model="facebook/blenderbot-400M-distill")

Device set to use cuda:0


In [4]:
user_message = """
What are some fun activities I can do in the winter?
"""

In [9]:
response = chatbot(user_message)
print(response)

[{'generated_text': " I like to go snowboarding.  It's a lot of fun.  What do you like to do in winter?"}]
